Артем Жук, 399 группа
### EM for PCA

В данном репозитории реализованы две вариации ЕМ-алогоритма для анализа главных компонент. Статья с описанием выкладок: http://www.machinelearning.ru/wiki/images/7/73/BMMO11_11.pdf

#### EMPCA
Класс EMPCA является реализацией ЕМ алгоритма для нахожения подпространста пространства признаков, порожденного первыми несколькими главными компонентами. Для всей выборки все признаки должны быть известны.  
Эта реализация опирается на достаточно простые выкладки, а кроме того быстрее работает, чем более общая реализация EMPCAM.

In [1]:
from empca import EMPCA
from empca_missing import EMPCAM
import numpy as np
from utils import random_model, gram_schmidt, span_in

Сгенерируем выборку. Метод random_model(N, D, d) генрирует выборку с распределением $W\mathcal{N}(0, I_d) + \mathcal{N}(0, \varepsilon I_D), W\in M(\mathbb{R})^{D\times d}$.  
gram_schmidt(X) применяет алгоритм Грама-Шмидта к строкам Х.

In [2]:
X, W, T = random_model(30, 6, 1)
print("W=\n", W)
print("X..=\n", X[:5])

W=
 [[ 5.97881136 -0.25410297  6.81030868  1.03159307  1.50557477  1.71962527]]
X..=
 [[ -1.0114197    0.58374804  -1.29083331   0.68349275   0.27928766
   -1.07266988]
 [  6.85317976  -0.15636167   6.30807068   0.38520335   2.57695996
    0.72779977]
 [ -3.14353193  -0.14074872  -2.98417836  -0.04047946  -0.17713702
   -1.15674936]
 [ -1.85105285  -0.09866784  -1.59239429  -0.1066606   -0.70153864
   -0.68210647]
 [ 11.69533048  -0.32631081  16.44522194   2.77762191   1.75889859
    2.11808923]]


Создадим экземпляр EMPCA и запустим на нашей выборке. Поле components_ содержит ортнормированный базис из n_components главных векторов.

In [3]:
empca = EMPCA(n_components=1, n_iter=100)
empca.fit(X)
print('basis:\n', empca.components_)
print('original basis:\n', gram_schmidt(W))

basis:
 [[-0.64085374  0.01528599 -0.72174713 -0.12485561 -0.14901317 -0.17424137]]
original basis:
 [[ 0.6356155  -0.02701403  0.72401311  0.10967005  0.16005969  0.18281568]]


Как видим, алгоритм довольно точно определил направляющий вектор прямой, на которой лежат точки. Попробуем для плоскости:

In [4]:
X, W, T = random_model(60, 7, 5)
empca = EMPCA(n_components=1, n_iter=100)
empca.fit(X)
print('basis:\n', empca.components_)
print('original basis:\n', gram_schmidt(W))

basis:
 [[ 0.3744648  -0.24705206  0.08836305  0.28322767 -0.54327397  0.11216289
   0.63481362]]
original basis:
 [[ 0.28218453 -0.61263134 -0.00523518 -0.14750588 -0.68357298  0.0082954
   0.2364921 ]
 [ 0.13499266 -0.42230095  0.31156154  0.47151671  0.11651167 -0.32128151
  -0.60600621]
 [ 0.71312337  0.27615395  0.50906223  0.17124248  0.1030607   0.20349402
   0.27330273]
 [-0.16219424 -0.2593355  -0.23291234  0.63926136  0.11106955  0.62061268
   0.21456546]
 [ 0.18183647 -0.24757823  0.00906431 -0.52293611  0.25918127  0.59741652
  -0.45608634]]


На этот раз сложнее проверить совпадение векторов. Однако для проверки корректности нам можно проверять не точное совпадение базисов, а совпадение подпростраств, на них натянутых. Для этого воспользуемся функцией span_in(A, B), проверяющей что строки А лежат в линейной оболочке строк B.

In [5]:
print('span_in: ', span_in(empca.components_, gram_schmidt(W)))

span_in:  True


#### EMPCAM
Этот класс реализует тот же алгоритм, но допускает пропуски в данных. 


Сгененрируем выборку с 20% пропущенных значений:

In [6]:
X, W, T = random_model(100, 10, 1, 0.2)
print("X=\n", X[:5])

X=
 [[  4.26285878e+00   3.88755828e+00   4.29777531e+00  -4.21641699e+00
   -2.70120058e+00              nan  -6.46463723e+00   8.72853671e-01
    9.69187537e-01              nan]
 [  2.71283340e+00   1.25288107e+00   7.07805781e-01  -1.83715203e+00
   -1.19871650e+00   1.97832064e+00  -1.74787508e+00   4.66906680e-01
    2.13373625e-01  -4.00713119e-01]
 [ -1.47403217e+01  -1.02856123e+01              nan   1.33227516e+01
    9.50989007e+00  -1.49178965e+01   1.73871753e+01  -5.09257789e+00
               nan              nan]
 [             nan  -1.79415038e-01  -7.41956211e-02   4.19260230e-01
    5.83203057e-01              nan  -5.09768295e-01   3.90888213e-01
               nan              nan]
 [  2.57178545e-01  -3.87874398e-01  -6.52189193e-02   1.47784103e-02
   -3.53184410e-02  -5.48074629e-01  -5.52581315e-01   6.23576397e-02
    6.10158782e-01  -2.63784016e-01]]


И применим к ней алгоритм: 

In [7]:
empcam = EMPCAM(n_components=1, n_iter=100)
empcam.fit(X)
A = empcam.components_
B = gram_schmidt(W)
print('found basis: ', A)
print('original basis: ', B)
d = A[0] - B[0]
print('A - B: ', d)
print('diff = ', np.linalg.norm(d))

found basis:  [[ 0.43681588  0.28054315  0.34246973 -0.3689935  -0.20429254  0.38358431
  -0.5024802   0.09531065  0.09550308 -0.13221962]]
original basis:  [[ 0.42308584  0.28057189  0.32287548 -0.38772854 -0.22494736  0.41527214
  -0.47321857  0.10403386  0.1120527  -0.13163909]]
A - B:  [  1.37300439e-02  -2.87451899e-05   1.95942496e-02   1.87350425e-02
   2.06548242e-02  -3.16878274e-02  -2.92616301e-02  -8.72320864e-03
  -1.65496238e-02  -5.80528747e-04]
diff =  0.0596720702019


Как видим, нам удалось довольно точно восстановить исходную зависимость. 

Теперь построим модель с большим числом латентных переменных и проверим, что найденное подпрастранство попадает в пространство, натянутое на образы латентных переменных.

In [8]:
X, W, T = random_model(100, 30, 5, 0.1)
empcam = EMPCAM(n_components=2, n_iter=200)
empcam.fit(X)
print('lies_inside: ', span_in(empcam.components_, gram_schmidt(W)))

lies_inside:  True
